# Implement some Python support functions

In [ ]:
import OpenVisus as ov

from PIL import Image
from urllib.request import urlopen
import matplotlib.pyplot as plt
import shutil
import numpy

def LoadImage(filename):
	if filename.startswith('http'):
		filename=urlopen(filename) 
	return numpy.asarray(Image.open(filename))

def ShowImage(data,width=10):
	ratio=float(data.shape[1])/data.shape[0]
	fig = plt.figure(figsize = (width,width*ratio))
	ax = fig.add_subplot(1,1,1)
	ax.imshow(data, origin='lower')
	plt.show()

db=ov.LoadDataset("https://s3.us-west-1.wasabisys.com/exxon_pari_jan2013-time/visus.idx")
print(db.getDatasetBody().toString())

# change as needed
access_key="XXXX"
secret_access_key="YYYY"

access= db.createAccessForBlockQuery(ov.StringTree.fromString("""
	<access type='multiplex'>
		<access  type='disk' chmod='rw' url='file://./visus_cache/exxon_pari_jan2013-time/visus.idx' />
		<access type="CloudStorageAccess"  chmod='r' compression='zip'
			url='https://s3.us-west-1.wasabisys.com/exxon_pari_jan2013-time/visus.idx?access_key={access_key="XXXX"}&amp;secret_key={secret_access_key}' 
			filename_template='/exxon_pari_jan2013-time/$(time)/$(field)/$(block:%016x)' />
	</access>
""".format(access_key=access_key,secret_access_key=secret_access_key)))

data=db.read(access=access, z=[512,513], time=0,quality=-6)

In [ ]:
import OpenVisus as ov

#dataset_url = f"file://E:/ExxonMobil/exxonmobile-time.idx"

# ///////////////////////////////////////////////////////////////////////////////////////
def ReadVisusSlice(db=None, x=None, y=None, z=None, time=None, field=None, access=access, resolution=-2, resample=False):

	"""" 
	Example:
																				 223  333
																				 890  123
		----------------------------------------------------------------
		bitmask V(012)(012)(012)(012)(012)(012)(012)(012)(012)(012)(012)

		z=[1000,1001],resolution=-0 -> H=33
		y=[1000,1001],resolution=-0 -> H= 33
		x=[1000,1001],resolution=-0 -> H= 33

		z=[1000,1001],resolution=-1 -> H= 32
		y=[1000,1001],resolution=-1 -> H= 31
		x=[1000,1001],resolution=-1 -> H= 30

		z=[1000,1001],resolution=-2 -> H= 29
		y=[1000,1001],resolution=-2 -> H= 28
		x=[1000,1001],resolution=-2 -> H= 27
	"""

	


	assert resolution<=0

	H=db.getMaxResolution()
	bitmask=db.db.getBitmask().toString()
	BOX,SIZE = db.getLogicBox(),db.getLogicSize()
	assert bitmask[0]=="V" and (len(bitmask)-1)==H

	bounds=[x,y,z]

	for A in range(3):

		if bounds[A] is None: 
			bounds[A]=[BOX[0][A],BOX[1][A]] # box is in the format [x1,y1,z1] [x2,y2,z2]
	
		if isinstance(bounds[A],int): 
			bounds[A]=[bounds[A],bounds[A]+1]

		if (bounds[A][1]-bounds[A][0])==1: 
			axis, offset=A, bounds[A][0]

		assert (bounds[A][1]-bounds[A][0])>=1

	# choose the max_resolution and align the offset
	factor=1
	for I in range(H,0,-1):
		
		# I want full resolution
		if resolution==0:
			H=I
			break

		# example: -1 means I want to 'jump' one Z bit in the bitmask (given the axis ==2)
		if ord(bitmask[I])-ord('0')==axis:
			resolution+=1
			factor*=2
			offset=(offset // factor) * factor
			bounds[axis][0]=offset
			bounds[axis][1]=offset+1

	logic_box=[
		[bounds[0][0], bounds[1][0], bounds[2][0]],
		[bounds[0][1], bounds[1][1], bounds[2][1]]
	]

#	print("bitmask",bitmask,"H",H,"axis",axis,"offset",offset)
	ret = db.read(logic_box=logic_box, max_resolution=H, time=time, field=field, access=access)

	# lower the dimension to match the pdim
	if axis==0: ret=ret[:,:,0]
	if axis==1: ret=ret[:,0,:]
	if axis==2: ret=ret[0,:,:]

	# resample
	if resample==True:
		X,Y=(axis+1) % 3,(axis+2) % 3
		X,Y=min(X,Y),max(X,Y)
		resample=[SIZE[X],SIZE[Y]]

	# resample
	if	isinstance(resample,(tuple,list)) :
		from skimage.transform import resize
		ret = resize(ret, resample, preserve_range=True).astype(ret.dtype)

	return ret


In [ ]:
import OpenVisus as ov

#dataset_url = f"file://E:/ExxonMobil/exxonmobile-time.idx"

# ///////////////////////////////////////////////////////////////////////////////////////
def ReadVisusSlice(db=None, x=None, y=None, z=None, time=None, 
                   field=None, access=access, resolution=-2, resample=False, boxOffset = 800):

	"""" 
	Example:
																				 223  333
																				 890  123
		----------------------------------------------------------------
		bitmask V(012)(012)(012)(012)(012)(012)(012)(012)(012)(012)(012)

		z=[1000,1001],resolution=-0 -> H=33
		y=[1000,1001],resolution=-0 -> H= 33
		x=[1000,1001],resolution=-0 -> H= 33

		z=[1000,1001],resolution=-1 -> H= 32
		y=[1000,1001],resolution=-1 -> H= 31
		x=[1000,1001],resolution=-1 -> H= 30

		z=[1000,1001],resolution=-2 -> H= 29
		y=[1000,1001],resolution=-2 -> H= 28
		x=[1000,1001],resolution=-2 -> H= 27
	"""

	#print(db, x, y, z, time, field, access, resolution, resample, boxOffset )



	assert resolution<=0

	H=db.getMaxResolution()
	bitmask=db.db.getBitmask().toString()
	BOX,SIZE = db.getLogicBox(),db.getLogicSize()
	assert bitmask[0]=="V" and (len(bitmask)-1)==H

	bounds=[x,y,z]
	min_box = boxOffset    
	max_box = boxOffset+400    

	for A in range(3):

		if bounds[A] is None: 
			bounds[A]=[min_box,max_box] # box is in the format [x1,y1,z1] [x2,y2,z2]
	
		if isinstance(bounds[A],int): 
			bounds[A]=[bounds[A],bounds[A]+1]

		if (bounds[A][1]-bounds[A][0])==1: 
			axis, offset=A, bounds[A][0]

		assert (bounds[A][1]-bounds[A][0])>=1

	# choose the max_resolution and align the offset
	factor=1
	for I in range(H,0,-1):
		
		# I want full resolution
		if resolution==0:
			H=I
			break

		# example: -1 means I want to 'jump' one Z bit in the bitmask (given the axis ==2)
		if ord(bitmask[I])-ord('0')==axis:
			resolution+=1
			factor*=2
			offset=(offset // factor) * factor
			bounds[axis][0]=offset
			bounds[axis][1]=offset+1

	logic_box=[
		[bounds[0][0], bounds[1][0], bounds[2][0]],
		[bounds[0][1], bounds[1][1], bounds[2][1]]
	]

#	print("bitmask",bitmask,"H",H,"axis",axis,"offset",offset)
	ret = db.read(logic_box=logic_box, max_resolution=H, time=time, field=field, access=access)

	# lower the dimension to match the pdim
	if axis==0: ret=ret[:,:,0]
	if axis==1: ret=ret[:,0,:]
	if axis==2: ret=ret[0,:,:]

	# resample
	if resample==True:
		X,Y=(axis+1) % 3,(axis+2) % 3
		X,Y=min(X,Y),max(X,Y)
		resample=[SIZE[X],SIZE[Y]]

	# resample
	if	isinstance(resample,(tuple,list)) :
		from skimage.transform import resize
		ret = resize(ret, resample, preserve_range=True).astype(ret.dtype)

	return ret


# Show other OpenVisus slices

In [ ]:

%matplotlib notebook
from ipywidgets import *
import numpy as np
import matplotlib.pyplot as plt

mysubplots = []

def ShowImages(images, ncols=3, single_width=3, color_map='gray'):
	"""
	Show images in a grid
	"""
	for I in range(len(images)):
		images[I]["img"]=images[I]["img"].astype(float)

	first=images[0]
	ratio=float(first["img"].shape[1])/first["img"].shape[0]
	single_height=single_width*ratio
	N=len(images)
	nrows=N//ncols + (1 if N % ncols else 0)
	fig = plt.figure(figsize=(single_width*ncols, single_height*nrows))
	print (single_width*ncols, single_height*nrows)
	for I in range(N):
		fig.add_subplot(nrows, ncols, I+1)
		#plt.imshow(images[I]["img"], cmap=plt.get_cmap(color_map))
		plt.axis('off')
		#plt.title(images[I]["title"])
		mysubplots.append(plt)
	return fig

type="s"
#db=ov.LoadDataset(dataset_url)

time_a=0 ;img_a=ReadVisusSlice(db=db, x=None, y=None,z=500, resolution=-4, time=time_a, access=access)
time_b=2 ;img_b=ReadVisusSlice(db=db, x=None, y=None,z=500, resolution=-4, time=time_b, access=access)

myfig = ShowImages([
	{"title": f"time-{time_a}","img": img_a},
	{"title": f"time-{time_b}","img": img_b},
	{"title": "diff","img":img_a-img_b}
])


myfig.add_subplot(1, 3,1)
plt.axis('off')
plt.subplots_adjust(wspace = 0)
plt.imshow(img_a)
myfig.add_subplot(1, 3,2)
plt.axis('off')
plt.subplots_adjust(wspace = 0)
plt.imshow(img_a)
myfig.add_subplot(1, 3,3)
plt.axis('off')
plt.subplots_adjust(wspace = 0)
plt.imshow(img_a)

import scipy.ndimage

old_type = None

def update(T1 = 1, T2 = 0, slice=500,axis=2,sigma=0, boxOffset=800, resolution=-3): #=347):
#    line.set_ydata(f(x,A,B,C))
#    print (T1,T2)
    global old_type, db
    if old_type != type:
        old_type = type
        if type == 'reconstruction':
            #db=ov.LoadDataset(dataset_url)
            pass
        else:
            #db=ov.LoadDataset(dataset_url)
            pass
        
    if axis ==0:
        time_a=0 ;img_a=ReadVisusSlice(db=db, x=slice, y=None,z=None, resolution=resolution, time=T1, boxOffset=boxOffset)
        time_b=60;img_b=ReadVisusSlice(db=db, x=slice, y=None,z=None, resolution=resolution, time=T2, boxOffset=boxOffset)
    if axis ==1:
        time_a=0 ;img_a=ReadVisusSlice(db=db, x=None, y=slice,z=None, resolution=resolution, time=T1, boxOffset=boxOffset)
        time_b=60;img_b=ReadVisusSlice(db=db, x=None, y=slice,z=None, resolution=resolution, time=T2, boxOffset=boxOffset)
    if axis ==2:
        time_a=0 ;img_a=ReadVisusSlice(db=db, x=None, y=None,z=slice, resolution=resolution, time=T1, boxOffset=boxOffset)
        time_b=60;img_b=ReadVisusSlice(db=db, x=None, y=None,z=slice, resolution=resolution, time=T2, boxOffset=boxOffset)
    img_a =scipy.ndimage.zoom(img_a, (500/img_a.shape[0],500/img_a.shape[1]), order = 2)
    img_b =scipy.ndimage.zoom(img_b, (500/img_b.shape[0],500/img_b.shape[1]), order = 2)
    #sigma = 2
    img_a = scipy.ndimage.gaussian_filter(img_a, sigma=sigma)
    img_b = scipy.ndimage.gaussian_filter(img_b, sigma=sigma)
    myfig.clf()
    myfig.add_subplot(1, 3,1)
    plt.axis('off')
    plt.subplots_adjust(wspace = 0)
    plt.imshow(img_a)
    myfig.add_subplot(1, 3,2)
    plt.axis('off')
    plt.subplots_adjust(wspace = 0)
    plt.imshow(img_b)
    myfig.add_subplot(1, 3,3)
    plt.axis('off')
    plt.subplots_adjust(wspace = 0)
    plt.imshow(img_a-img_b)


In [ ]:
interact(update, T1 = (0,99,1), T2 = (0,99,1), slice =(0,1500,1), 
         axis=(0,2,1),sigma=(0,5,0.001),boxOffset=(0,5000,1),resolution=(-9,0,1),
         layout=widgets.Layout(width='30%'));